In [5]:
import pandas as pd 
pay_bill = pd.read_csv('data/credit_record.csv')
pay_bill

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [6]:
def get_unused_user(df):
    df = df.groupby('ID').apply(lambda x: x['STATUS'].isin(['X']).all()).reset_index()
    df = df[df[0] == True]
    df.rename(columns={0: 'credit_decision'}, inplace=True)
    df['credit_decision'] = 0
    return df
unused_df = get_unused_user(pay_bill)
unused_df

,ID,credit_decision
2,5001713,0
3,5001714,0
4,5001715,0
16,5001731,0
17,5001732,0
...,...,...
45947,5150409,0
45948,5150410,0
45965,5150460,0
45979,5150481,0


In [7]:
credit_decision = pd.DataFrame()
used_df = pay_bill[pay_bill['STATUS'] != 'X']
credit_decision['ID'] = used_df['ID'].unique()

def get_used_credit_decision(id):
    '''Determine for a given df, if the client is good, bad or unused status according to the credit usage.'''
    client_status = used_df[used_df['ID'] == id]['STATUS'].tolist()
    c_count = client_status.count('C')
    total_count = len(client_status)
    if c_count / total_count >= 0.7:
        return 1
    else:
        return -1

credit_decision['credit_decision'] = credit_decision['ID'].map(get_used_credit_decision)
credit_decision

,ID,credit_decision
0,5001711,-1
1,5001712,-1
2,5001717,-1
3,5001718,-1
4,5001719,1
...,...,...
41444,5150480,1
41445,5150482,-1
41446,5150484,-1
41447,5150485,-1


In [8]:
credit_decision['credit_decision'].value_counts()

credit_decision
-1    29402
 1    12047
Name: count, dtype: int64

In [9]:
final_decision = pd.concat([unused_df, credit_decision])
final_decision

,ID,credit_decision
2,5001713,0
3,5001714,0
4,5001715,0
16,5001731,0
17,5001732,0
...,...,...
41444,5150480,1
41445,5150482,-1
41446,5150484,-1
41447,5150485,-1


In [11]:
# final_decision.to_csv('data/credit_decision.csv', index=False)

In [12]:
app_record = pd.read_csv('data/application_record.csv')
app_record = pd.merge(app_record, final_decision, on='ID', how='inner')
app_record

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,credit_decision
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,1,0,0,0,Managers,2.0,-1
36453,5149834,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-1
36454,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-1
36455,5150049,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,1,0,0,0,Sales staff,2.0,-1


In [16]:
app_record['NAME_EDUCATION_TYPE'].value_counts()

NAME_EDUCATION_TYPE
Secondary / secondary special    24777
Higher education                  9864
Incomplete higher                 1410
Lower secondary                    374
Academic degree                     32
Name: count, dtype: int64

In [18]:
income = pd.read_csv('data/income_train.csv')
income['education'].value_counts()

education
HS-grad         14197
Some-college     9790
Bachelors        7219
Masters          2392
Assoc-voc        1831
11th             1647
Assoc-acdm       1447
10th             1250
7th-8th           862
Prof-school       748
9th               684
12th              587
Doctorate         536
5th-6th           468
1st-4th           223
Preschool          76
Name: count, dtype: int64